In [13]:
%display latex

In [14]:
import algorithm_states, common

We start with a $ n \times n $-matrix $ m $.

In [15]:
l = 10
n = 3
m0 = matrix(SL(n, Integers(l)).random_element()); m0

[2 9 7]
[1 4 7]
[2 3 8]

In [26]:
state = algorithm_states.SLnState(m0)

Note that we can lift elements of $ \mathbb Z / l \mathbb Z $ to $ \mathbb Z $ such that they end up between $ 0 $ and $ l - 1 $.
Take $ i $ and $ j $ such that the lift of $ m_{1i} $ is less than or equal to $ m_{1j} $. Then subtract $ m_{1i} $ from $ m_{1j} $. This decreases the sum of their lifts. Since this sum is finite, if we do this repeatedly, all but one of the $ m_{1j} $ will become $ 0 $ in a finite number of steps.

In [27]:
while len([i for i in range(n) if state.m[0, i] != 0]) > 1:
    columns = sorted([(ZZ(state.m[0, i]), i) for i in range(n)], reverse = True)
    source = columns[1][1]
    target = columns[0][1]
    factor = columns[0][0] // columns[1][0]
    state.add_column(source, target, scale = -factor, description = f"Subtract {factor} times column {source} from {target}")

Subtract 1 times column 2 from 1


[2 2 7]
[1 7 7]
[2 5 8]

Subtract 3 times column 1 from 2


[2 2 1]
[1 7 6]
[2 5 3]

Subtract 1 times column 0 from 1


[2 0 1]
[1 6 6]
[2 3 3]

Subtract 2 times column 2 from 0


[0 0 1]
[9 6 6]
[6 3 3]

Let $ m_{1i} $ be the nonzero value. If $ i \not = n $, we add $ m_{1i} $ to $ m_{1n} $ and then subtract $ m_{1n} $ from $ m_{1i} $.

In [28]:
if state.m[0, n - 1] == 0:
    i = [i for i in range(n) if state.m[0, i] != 0][0]
    state.switch_columns(i, n - 1)

Let $ a $ be the determinant of the bottom left $ (n-1) \times (n-1) $ submatrix. Note that $ m_{1n} a = 1 $. Add $ a $ times $ m_{1n} $ to $ m_{11} $, which makes sure that $ m_{11} = 1 $. Then subtract $ m_{1n} $ times $ m_{11} $ from $ m_{1n} $. Also, subtract $ m_{i1} $ times $ m_{11} $ from $ m_{i1} $ for all $ i \geq 2 $.

In [29]:
state.add_column(n - 1, 0, scale = ZZ(1 / state.m[0, n - 1]))

[1 0 1]
[5 6 6]
[9 3 3]

In [30]:
state.add_column(0, n - 1, scale = -state.m[0, n - 1])

[1 0 0]
[5 6 1]
[9 3 4]

In [31]:
for i in range(1, n):
    state.add_row(0, i, scale = -state.m[i, 0], description = f"Make entry (0, {i}) equal to 0")

Make entry (0, 1) equal to 0


[1 0 0]
[0 6 1]
[9 3 4]

Make entry (0, 2) equal to 0


[1 0 0]
[0 6 1]
[0 3 4]

Note that the bottom right $ (n - 1) \times (n - 1) $ submatrix is an element of $ \mathop{SL}_{n-1}(\mathbb Z / l \mathbb Z) $. Repeating the above for this submatrix (induction) gives the identity matrix.

In [32]:
for j in range(1, n - 1):
    while len([i for i in range(n) if state.m[j, i] != 0]) > 1:
        columns = sorted([(ZZ(state.m[j, i]), i) for i in range(n)], reverse = True)
        source = columns[1][1]
        target = columns[0][1]
        factor = columns[0][0] // columns[1][0]
        state.add_column(source, target, scale = -factor, description = f"Subtract {factor} times column {source} from {target}")
        
    if state.m[j, n - 1] == 0:
        i = [i for i in range(n) if state.m[j, i] != 0][0]
        state.switch_column(i, n - 1, description = f"Switch columns {i} and {n - 1}")
        
    state.add_column(n - 1, j, scale = ZZ(1 / state.m[j, n - 1]), description = f"Make entry ({j}, {j}) equal to 1")
    
    state.add_column(j, n - 1, scale = -state.m[j, n - 1], description = f"Make entry ({j}, {n - 1}) equal to 0")
    
    for i in range(j + 1, n):
        state.add_row(j, i, scale = -state.m[i, j], description = f"Make entry ({i}, {j}) equal to zero")

Subtract 6 times column 2 from 1


[1 0 0]
[0 0 1]
[0 9 4]

Make entry (1, 1) equal to 1


[1 0 0]
[0 1 1]
[0 3 4]

Make entry (1, 2) equal to 0


[1 0 0]
[0 1 0]
[0 3 1]

Make entry (2, 1) equal to zero


[1 0 0]
[0 1 0]
[0 0 1]

In [33]:
left_lift, right_lift = state.t; state.t

(
[ 1  0  0]  [    1   -46  -405]
[ 5  1  0]  [    3  -119 -1047]
[36  7  1], [   -4   159  1399]
)

In [34]:
(left_lift.inverse() * right_lift.inverse())

[  -8  -41  -33]
[  31  -16   -3]
[  72 1613 1228]

In [35]:
(m0, (left_lift.inverse() * right_lift.inverse()).change_ring(Integers(l)))

(
[2 9 7]  [2 9 7]
[1 4 7]  [1 4 7]
[2 3 8], [2 3 8]
)